In [1]:
import MC_functions as mc
import numpy as np
import matplotlib.pyplot as plt

In [13]:
# Simulation parameters
number_of_variables = 1
beta = np.array([2]) # in the multivariate case, just add elements until the size of the vector is equal to number_of_variables
N1 = 5
N2 = 30
T = 6
n = N1 * N2 * T
mu_epsilon = 0
sigma_epsilon = 8
non_zero_prob = 0.1
mu_x = 0
sigma_x = 1
mu_e = 0
sigma_e = 1
seed = 42

In [14]:
# Generate sparce covariance matrix
diagonal_vector, cov_matrix = mc.generate_cov_matrix(mu_epsilon, sigma_epsilon, n, non_zero_prob, seed)

In [15]:
# Sanity check
mc.is_positive_semidefinite(cov_matrix)

True

In [17]:
# Step 1: Generate x variables
x = np.random.normal(mu_x, sigma_x, (n, number_of_variables))

# Step 2: Generate three fixed-effect variables
fixed_effect_1 = np.random.normal(-2, 1, (N1, 1))
fixed_effect_2 = np.random.normal(3, 1, (N2, 1))
fixed_effect_3 = np.random.normal(1, 1, (T, 1))

# Step 3: Repeat the fixed-effect variables to match the desired dimensions
fixed_effect_1 = np.repeat(fixed_effect_1, N2 * T, axis=0)
fixed_effect_2 = np.tile(fixed_effect_2, (N1 * T, 1))
fixed_effect_3 = np.tile(fixed_effect_3, (N1 * N2, 1))

# Step 4: Stack the fixed-effect variables and combine x and fixed effects
fixed_effects_matrix = np.hstack((fixed_effect_1, fixed_effect_2, fixed_effect_3))
X = np.hstack((x, fixed_effects_matrix))

# Step 5: generate disturbances
disturbances = np.random.normal(mu_e, sigma_e, (n, 1))

# Step 6: Generate y using linear relationship: y = X * beta + FE + disturbances
y = np.dot(x, beta) + fixed_effects_matrix.sum(axis=1) + disturbances.flatten()

# Step 7: Estimate beta using OLS formula
X_with_intercept = np.hstack((np.ones((n, 1)), X))  # Adding intercept term
X_transpose = np.transpose(X_with_intercept)
X_transpose_X_inv = np.linalg.inv(np.dot(X_transpose, X_with_intercept))
beta_hat = np.dot(np.dot(X_transpose_X_inv, X_transpose), y)

In this initial version we use iid disturbances
Alternatively, we can also use the ```generate_serially_correlated_disturbances``` function to generate serially correlated disturbances. The function takes in the following parameters: ```(mu_e, sigma_e, n, N1, N2, T dim_to_correlate, seed)```
The function generates disturbances and introduces serial correlation based on the specified dimension ```dim_to_correlate```. The serial correlation is implemented by adding past disturbances to current disturbances, creating a correlation pattern along the chosen dimension.
The dim_to_correlate parameter can be set to either "N1", "N2", or "T" to choose the dimension along which you want to introduce serial correlation.

Functions to be writtem: $\varepsilon \sim N(0, \Omega)$, $\varepsilon \sim MT(\Omega, 0, 5)$, a multivariate t-distribution of degree 5 and $\varepsilon \sim SN(0, \Sigma, \Lambda)$ a skewed-Normal distribution.